# Keto/Vegan Diet classifier
Argmax, a consulting firm specializing in search and recommendation solutions with offices in New York and Israel, is hiring entry-level Data Scientists and Machine Learning Engineers.

At Argmax, we prioritize strong coding skills and a proactive, “get-things-done” attitude over a perfect resume. As part of our selection process, candidates are required to complete a coding task demonstrating their practical abilities.

In this task, you’ll work with a large recipe dataset sourced from Allrecipes.com. Your challenge will be to classify recipes based on their ingredients, accurately identifying keto (low-carb) and vegan (no animal products) dishes.

Successfully completing this assignment is a crucial step toward joining Argmax’s talented team.

In [1]:
from opensearchpy import OpenSearch
from decouple import config
import pandas as pd

client = OpenSearch(
    hosts=[config('OPENSEARCH_URL', 'http://localhost:9200')],
    http_auth=None,
    use_ssl=False,
    verify_certs=False,
    ssl_show_warn=False,
)

# Recipes Index
Our data is stored in OpenSearch, and you can query it using either Elasticsearch syntax or SQL.
## Elasticsearch Syntax

In [2]:
query = {
    "query": {
        "match": {
            "description": { "query": "egg" }
        }
    }
}

res = client.search(
    index="recipes",
    body=query,
    size=2
)

hits = res['hits']['hits']
hits

[{'_index': 'recipes',
  '_id': 'qx5OVpcBzHS7qnWEfscu',
  '_score': 3.9817066,
  '_source': {'title': 'Genuine Egg Noodles',
   'description': 'These egg noodles are the original egg noodles.  ',
   'instructions': ['Combine flour, salt and baking powder. Mix in eggs and enough water to make the dough workable. Knead dough until stiff. Roll into ball and cut into quarters. Using 1/4 of the dough at a time, roll flat to about 1/8 inch use flour as needed, top and bottom, to prevent sticking. Peel up and roll from one end to the other. Cut roll into 3/8 inch strips. Noodles should be about 4 to 5 inches long depending on how thin it was originally flattened. Let dry for 1 to 3 hours.',
    'Cook like any pasta or, instead of drying first cook it fresh but make sure water is boiling and do not allow to stick. It takes practice to do this right.'],
   'ingredients': ['2 cups Durum wheat flour',
    '1/2 teaspoon salt',
    '1/4 teaspoon baking powder',
    '3 eggs',
    'water as needed'],

## SQL syntax

In [3]:
query = """
SELECT *
FROM recipes
WHERE description like '%egg%'
LIMIT 10
"""

res = client.sql.query(body={'query': query})
df = pd.DataFrame(res["datarows"], columns=[c["name"] for c in res["schema"]])
df

,description,ingredients,instructions,photo_url,title
0,"Thumbprint cookies with a delicious filling, t...","[3/4 cup butter, softened, 1/2 cup white sugar...",[Preheat the oven to 350 degrees F (175 degree...,http://images.media-allrecipes.com/userphotos/...,Eggnog Thumbprints
1,This is my Grandma's recipe -- very light in c...,"[2 cups white sugar, 1 cup shortening, 2 teasp...","[Mix together the sugar, shortening, baking po...",http://images.media-allrecipes.com/userphotos/...,Grandma Sheets' Sugar Cookies
2,"This cookie recipe doesn't need eggs, so you c...","[2 tablespoons peanut butter, 1 cup white suga...",[Preheat oven to 375 degrees F (190 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Super Dooper Peanut Butter
3,"These cookies can be crisp, or chewy. If you ...","[1 cup packed brown sugar, 1 cup white sugar, ...","[Mix sugars, peanut butter, and shortening. Ad...",http://images.media-allrecipes.com/userphotos/...,Chewy Peanut Butter Cookies
4,The French word sables means 'sand cookies' or...,"[3 eggs, 2 1/3 cups all-purpose flour, 1/2 cup...",[Place eggs (and water to cover) in a medium s...,http://images.media-allrecipes.com/userphotos/...,Rochelle Sable
5,This is a really easy recipe that uses no oil ...,[1 (3.9 ounce) package instant chocolate puddi...,[Preheat oven to 350 degrees F (175 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Easy Chocolaty Brownies
6,"A rich, crisp, sugar type cookie that is easy ...","[1 cup white sugar, 2 cups all-purpose flour, ...","[Mix together sugar, flour, baking soda, and s...",http://images.media-allrecipes.com/userphotos/...,Mayonnaise Cookies
7,"Aebleskiver - a Danish dessert, like doughnut ...","[2 egg whites, 2 cups all-purpose flour, 2 tea...","[In a clean glass or metal bowl, beat the egg ...",http://images.media-allrecipes.com/userphotos/...,Aebleskiver
8,Here's a terrific recipe for icing sugar cooki...,"[1 1/2 cups confectioners' sugar, 3 tablespoon...","[Combine confectioners' sugar butter, vanilla ...",http://images.media-allrecipes.com/userphotos/...,Butter Icing for Sugar Cookies
9,An easy to make egg-free frosting that works g...,"[1/2 cup butter, softened, 1/2 cup shortening,...","[In a large bowl, cream together the butter, s...",http://images.media-allrecipes.com/userphotos/...,Buttercream Icing


# Task Instructions

Your goal is to implement two classifiers:

1.	Vegan Meal Classifier
1.	Keto Meal Classifier

Unlike typical supervised machine learning tasks, the labels are not provided in the dataset. Instead, you will rely on clear and verifiable definitions to classify each meal based on its ingredients.

### Definitions:

1. **Vegan Meal**: Contains no animal products whatsoever (no eggs, milk, meat, etc.).
1. **Keto Meal**: Contains no ingredients with more than 10g of carbohydrates per 100g serving. For example, eggs are keto-friendly, while apples are not.

Note that some meals may meet both vegan and keto criteria (e.g., meals containing avocados), though most meals typically fall into neither category.

## Example heuristic:

In [4]:
def is_ingredient_vegan(ing):
    for animal_product in "egg meat milk butter veel lamb beef chicken sausage".split():
        if animal_product in ing:
            return False
    return True

def is_vegan_example(ingredients):
    return all(map(is_ingredient_vegan, ingredients))
    
df["vegan"] = df["ingredients"].apply(is_vegan_example)
df

,description,ingredients,instructions,photo_url,title,vegan
0,"Thumbprint cookies with a delicious filling, t...","[3/4 cup butter, softened, 1/2 cup white sugar...",[Preheat the oven to 350 degrees F (175 degree...,http://images.media-allrecipes.com/userphotos/...,Eggnog Thumbprints,False
1,This is my Grandma's recipe -- very light in c...,"[2 cups white sugar, 1 cup shortening, 2 teasp...","[Mix together the sugar, shortening, baking po...",http://images.media-allrecipes.com/userphotos/...,Grandma Sheets' Sugar Cookies,False
2,"This cookie recipe doesn't need eggs, so you c...","[2 tablespoons peanut butter, 1 cup white suga...",[Preheat oven to 375 degrees F (190 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Super Dooper Peanut Butter,False
3,"These cookies can be crisp, or chewy. If you ...","[1 cup packed brown sugar, 1 cup white sugar, ...","[Mix sugars, peanut butter, and shortening. Ad...",http://images.media-allrecipes.com/userphotos/...,Chewy Peanut Butter Cookies,False
4,The French word sables means 'sand cookies' or...,"[3 eggs, 2 1/3 cups all-purpose flour, 1/2 cup...",[Place eggs (and water to cover) in a medium s...,http://images.media-allrecipes.com/userphotos/...,Rochelle Sable,False
5,This is a really easy recipe that uses no oil ...,[1 (3.9 ounce) package instant chocolate puddi...,[Preheat oven to 350 degrees F (175 degrees C)...,http://images.media-allrecipes.com/userphotos/...,Easy Chocolaty Brownies,False
6,"A rich, crisp, sugar type cookie that is easy ...","[1 cup white sugar, 2 cups all-purpose flour, ...","[Mix together sugar, flour, baking soda, and s...",http://images.media-allrecipes.com/userphotos/...,Mayonnaise Cookies,True
7,"Aebleskiver - a Danish dessert, like doughnut ...","[2 egg whites, 2 cups all-purpose flour, 2 tea...","[In a clean glass or metal bowl, beat the egg ...",http://images.media-allrecipes.com/userphotos/...,Aebleskiver,False
8,Here's a terrific recipe for icing sugar cooki...,"[1 1/2 cups confectioners' sugar, 3 tablespoon...","[Combine confectioners' sugar butter, vanilla ...",http://images.media-allrecipes.com/userphotos/...,Butter Icing for Sugar Cookies,False
9,An easy to make egg-free frosting that works g...,"[1/2 cup butter, softened, 1/2 cup shortening,...","[In a large bowl, cream together the butter, s...",http://images.media-allrecipes.com/userphotos/...,Buttercream Icing,False


### Limitations of the Simplistic Heuristic

The heuristic described above is straightforward but can lead to numerous false positives and negatives due to its reliance on keyword matching. Common examples of incorrect classifications include:
- "Peanut butter" being misclassified as non-vegan, as “butter” is incorrectly assumed to imply dairy.
- "eggless" recipes being misclassified as non-vegan, due to the substring “egg.”
- Animal-derived ingredients such as “pork” and “bacon” being incorrectly identified as vegan, as they may not be explicitly listed in the keyword set.


# Submission
## 1. Implement Diet Classifiers
Complete the two classifier functions in the diet_classifiers.py file within this repository. Ensure your implementation correctly identifies “keto” and “vegan” meals. After implementing these functions, verify that the Flask server displays the appropriate badges (“keto” and “vegan”) next to the corresponding recipes.

> **Note**
>
> This repo contains two `diet_classifiers.py` files:
> 1. One in this folder (`nb/src/diet_classifiers.py`)
> 2. One in the Flask web app folder (`web/src/diet_classifiers.py`)
>
> You can develop your solution here in the notebook environment, but to apply your solution 
> to the Flask app you will need to copy your implementation into the `diet_classifiers.py` 
> file in the Flask folder!!!

## EDA of dataset

In [10]:
df["desc_contains_vegan"] = df["description"].apply(lambda x: "vegan" in x.lower())

In [11]:
df["desc_contains_keto"] = df["description"].apply(lambda x: "keto" in x.lower())

In [5]:
def is_ingredient_keto(ingredient):
    # TODO: complete
    return False

def is_ingredient_vegan(ingredient):
    # TODO: complete
    return False    

For your convenience, you can sanity check your solution on a subset of labeled recipes by running `diet_classifiers.py`

In [8]:
! python diet_classifiers.py --ground_truth /usr/src/data/ground_truth_sample.csv

===Keto===
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
              precision    recall  f1-

## 2. Repository Setup
Create a **private** GitHub repository for your solution, and invite the GitHub user `argmax2025` as a collaborator. **Do not** share your implementation using a **forked** repository.

## 3. Application Form
Once you’ve completed the implementation and shared your private GitHub repository with argmax2025, please fill out the appropriate application form:
1. [US Application Form](https://forms.clickup.com/25655193/f/rexwt-1832/L0YE9OKG2FQIC3AYRR)
2.  [IL Application Form](https://forms.clickup.com/25655193/f/rexwt-1812/IP26WXR9X4P6I4LGQ6)


Your application will not be considered complete until this form is submitted.

## Evaluation process


Your submission will be assessed based on the following criteria:


1.	**Readability & Logic** – Clearly explain your approach, including your reasoning and any assumptions. If you relied on external resources (e.g., ingredient databases, nutrition datasets), be sure to cite them.
2.	**Executability** – Your code should run as is when cloned from your GitHub repository. Ensure that all paths are relative, syntax is correct, and no manual setup is required.
3.	**Accuracy** – Your classifiers will be evaluated against a holdout set of 20,000 recipes with verified labels. Performance will be compared to the ground truth.
data.


## Next steps
If your submission passes the initial review, you’ll be invited to a 3-hour live coding interview, where you’ll be asked to extend and adapt your solution in real time.

Please make sure you join from a quiet environment and have access to a Python-ready workstation capable of running your submitted project.